In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 로드 및 추출

- 네이버 크롤링 적금상품 데이터

In [5]:
df = pd.read_csv("/content/drive/MyDrive/ASAC/final/Saving.csv")

In [6]:
term_list = df['기간'].unique()

In [7]:
# 기간정하기
short_term_list = ['31일','200일', '1개월', '6개월미만','6개월만기', '3개월','100일','2개월','5개월']

In [8]:
short_tmp_3 = set()
for i in term_list:
    for s in short_term_list:
        idx = str(i).find(s)
        if idx != -1 :
            # 첫 시작일때만 확인해서 추가해주기 (11개월 같은거 추가 못하게)
            if idx == 0:
                short_tmp_3.add(i)

In [9]:
# 예외적인거 확인 후 추가해주기
short_tmp_3.add('03개월이상 ~ 06개월미만')
short_tmp_3.add('3~5개월')

In [10]:
# 리스트로변경
short_tmp_3 = list(short_tmp_3)

In [11]:
# 단기인지 아닌지 확인
df['단기'] = df['기간'].apply(lambda x : 1 if x in short_tmp_3 else 0)
short_df = df[df['단기'] == 1].reset_index(drop= True)

In [12]:
# 쓸데없는 컬럼 (모두 na 값) 날리기
short_df = short_df.dropna(axis = 'columns', how = 'all')

In [13]:
# 저장
short_df.to_csv("short_term_df.csv", encoding = 'utf-8-sig')

# 데이터 전처리

- 모두 수작업

- 우대사항 전처리
    - 우리 형식에 맞도록
- 원 데이터 프레임 형태와 동일시하게 변경
    - 컬럼명 및 형식
- 이자계산방식 변경
    - 만기일시지급 => 복리
    - 월지급식 => 단리
- 가입방법
    - 인터넷스마트폰전용
    - 인터넷스마트폰영업점
    - 영업점방문
- 금리 처리
    - 원래 크롤링되어 있는 것은 12개월 기준이라서 사용할 수 없음.
    - 기본 금리와 우대사항 정리한 것을 바탕으로 최대우대금리 정리

In [14]:
# 전처리 후 데이터 프레임
df_short = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/상품데이터_크롤링/적금/단기상품처리 - 정리본.csv")
df_short.head(2)

,금융회사,상품명,적립방식,세전이자율,세후이자율,세후이자(예시),최고우대금리,실질이자율,이자계산방식,기간_,비교공시일,담당부서 및 연락처,우대조건,가입방법,만기후 이자율,기타유의사항,등급,cluster_num,우대사항정리
0,카카오뱅크,카카오뱅크 한달적금,자유적립식,2.50%,NaN,NaN,8.00%,NaN,복리,31일,NaN,NaN,이적금에 납입하고 만기 해지한 경우 매일 우대금리와 보너스 우대금리를 합하여 최대 ...,인터넷스마트폰전용,NaN,NaN,NaN,NaN,* 만기 : 0.1% [매 입금시마다 연 0.1% 연 최대 3.1% 가능]\n* 이...
1,경남은행,오늘도 세이브 적금\n 네이버 페이,자유적립식,3.45%,NaN,NaN,6.50%,NaN,복리,6개월이내,NaN,NaN,"조건별 \n아래의 우대조건 충족 시, 최고 연 3.05% 우대이율을 적...",인터넷스마트폰전용,NaN,NaN,NaN,NaN,* 마케팅동의 : 0.15% [마케팅동의 및 모바일메세지수신 동의(신규시 제공)]\...


# 변수 추가
- 실질이자율
    - 기간이 명확하지 않아서...
- 등급
- 우대사항 정리

- 등급

In [15]:
bank_info_df = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/금융감독원데이터/경영지표/bank_grade.csv",encoding = 'cp949')

In [16]:
grade_dict = {
    'OSB저축은행': '오에스비저축은행',
    'JT친애저축은행': '제이티친애저축은행',
    'OK저축은행': '오케이저축은행',
    'MS저축은행': '엠에스상호저축은행',
    'DH저축은행': '디에이치저축은행',
    'JT저축은행': '제이티저축은행',
    'BNK저축은행': '비엔케이저축은행',
    '푸른저축은행': '푸른상호저축은행',
    'SBI저축은행': '에스비아이저축은행',
    '유니온저축은행': '유니온상호저축은행',
    'IBK저축은행': '아이비케이저축은행',
    '세람저축은행': '세람상호저축은행',
    'KB저축은행': '케이비저축은행',
    '대원저축은행': '대원상호저축은행',
    '카카오뱅크' : '주식회사 카카오뱅크',
    'KB국민은행' : '국민은행',
    'IBK기업은행' : '중소기업은행',
    '토스뱅크' : '토스뱅크 주식회사',
    'NH농협은행' : '농협은행주식회사',
    '케이뱅크' : '주식회사 케이뱅크',
    '평택저축은행' : '평택상호저축은행',
    'KDB산업은행' : '한국산업은행',
    'SNT저축은행' : '에스앤티저축은행'
}

In [17]:
def bank_grade(df):
    df['금융회사'] = df['금융회사'].apply(lambda x: grade_dict[x]if x in grade_dict.keys() else x )
    df = pd.merge(left = df, right = bank_info_df[['구분','등급']], how = 'left',
            left_on = '금융회사' , right_on = "구분")
    df.drop(columns = '구분', inplace = True)
    return df

In [18]:
col_list = list(df_short.columns)
col_list

['금융회사',
 '상품명',
 '적립방식',
 '세전이자율',
 '세후이자율',
 '세후이자(예시)',
 '최고우대금리',
 '실질이자율',
 '이자계산방식',
 '기간_',
 '비교공시일',
 '담당부서 및 연락처',
 '우대조건',
 '가입방법',
 '만기후 이자율',
 '기타유의사항',
 '등급',
 'cluster_num',
 '우대사항정리']

In [19]:
df_short.drop(columns = '등급', inplace = True)

In [20]:
df_short = bank_grade(df_short)

In [21]:
for i in range(len(df_short)):
    gr = df_short.loc[i, '등급']
    if np.isnan(gr) == True:
        print(df_short.loc[i, '금융회사'])

In [22]:
df_short = df_short[col_list]

- 우대사항정리

In [23]:
df_short['우대사항정리']

0      * 만기 : 0.1% [매 입금시마다 연 0.1% 연 최대 3.1% 가능]\n* 이...
1      * 마케팅동의 : 0.15% [마케팅동의 및 모바일메세지수신 동의(신규시 제공)]\...
2      * 최초거래 : 0.1% [가입시점에 우리은행 정기예금 및 적금 상품을 보유하지 않...
3      * 최초거래 : 0.1% [가입시점에 우리은행 정기예금 및 적금 상품을 보유하지 않...
4      * 최초거래 : 0.1% [가입시점에 우리은행 정기예금 및 적금 상품을 보유하지 않...
                             ...                        
106                                                  NaN
107                                                  NaN
108                                                  NaN
109                                                  NaN
110                                                  NaN
Name: 우대사항정리, Length: 111, dtype: object

In [24]:
df_short['우대사항정리'] = df_short['우대사항정리'].fillna('우대사항없음')

In [25]:
# 대분류 확인하기
cate_set = set()

for i in range(len(df_short)):
    text = df_short.loc[i,'우대사항정리' ]
    text_list = text.split("\n")
    for t in text_list :
    # 대분류
        cate = t.split(":")[0].replace("*","").strip()
        cate_set.add(cate)
cate_set

{'기타',
 '나이',
 '마케팅동의',
 '만기',
 '비대면',
 '우대사항없음',
 '이벤트',
 '이체횟수',
 '자동이체',
 '자동이체 0.1% [ 계약기간 해당 월차에 모두 자동이체할 경우 우대이율 적용]',
 '최초거래',
 '카드실적',
 '해당은행',
 '환경'}

In [26]:
# 다른 값 통일시켜주기 - 마케팅 동의
df_short['우대사항정리'] = df_short['우대사항정리'].apply(lambda x : x.replace('자동이체 0.1% ', "자동이체 : 0.1% ") if "자동이체 0.1% " in x else x)
df_short['우대사항정리'] = df_short['우대사항정리'].apply(lambda x : x.replace('3% ', "3.0%") if "3% " in x else x)

In [27]:
col_list_all_prefer = ['급여이체','마케팅동의','자동이체','카드실적','최초거래','주택청약',
                '비대면','영업점','해당은행','이체횟수','만기','변동금리','이벤트','기타',
                '나이','환경','신혼','반려인','직장인','여행']
col_list_all = ['금융회사', '상품명', '적립방식', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리','실질이자율',
       '이자계산방식', '기간', '비교 공시일', '담당부서 및 연락처', '우대조건', '가입대상.1', '가입방법', '만기후 이자율', '기타유의사항','등급', 'cluster_num','우대사항정리'] + col_list_all_prefer

In [28]:
# 모든 리스트에 따라 df의 col생성
for c in col_list_all_prefer:
    df_short[c] = np.nan

In [29]:
# 금리 추출
p = re.compile("[0-9]+[.][0-9]+[%]")


for i in range(len(df_short)):
    text = df_short.loc[i,'우대사항정리' ]
    if text == "우대사항없음":
        continue
    text_list = text.split("\n") # 우대사항 text 가지고오기
    for t in text_list :
        # 카테고리
        cate = t.split(":")[0].replace("*","").strip()
        # 금리
        rate = p.findall(t.split(":")[1])[0]
        df_short.loc[i,cate ] = rate

In [30]:
df_short.columns

Index(['금융회사', '상품명', '적립방식', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리', '실질이자율',
       '이자계산방식', '기간_', '비교공시일', '담당부서 및 연락처', '우대조건', '가입방법', '만기후 이자율',
       '기타유의사항', '등급', 'cluster_num', '우대사항정리', '급여이체', '마케팅동의', '자동이체',
       '카드실적', '최초거래', '주택청약', '비대면', '영업점', '해당은행', '이체횟수', '만기', '변동금리',
       '이벤트', '기타', '나이', '환경', '신혼', '반려인', '직장인', '여행'],
      dtype='object')

In [31]:
df_short.rename(columns = {'기간_' : '기간'}, inplace = True)

In [32]:
df_short.to_csv("./short_term_saving.csv", encoding = 'utf-8-sig')

In [33]:
df_short

,금융회사,상품명,적립방식,세전이자율,세후이자율,세후이자(예시),최고우대금리,실질이자율,이자계산방식,기간,...,만기,변동금리,이벤트,기타,나이,환경,신혼,반려인,직장인,여행
0,주식회사 카카오뱅크,카카오뱅크 한달적금,자유적립식,2.50%,NaN,NaN,8.00%,NaN,복리,31일,...,0.1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,경남은행,오늘도 세이브 적금\n 네이버 페이,자유적립식,3.45%,NaN,NaN,6.50%,NaN,복리,6개월이내,...,3.0%,NaN,NaN,3.0%,NaN,NaN,NaN,NaN,NaN,NaN
2,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,31일,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,100일,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,200일,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,영진저축은행,자유적립예금,자유적립식,1.20%,NaN,NaN,1.20%,NaN,복리,3개월이상 ~ 6개월이내,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,라온저축은행,자유적립예금,자유적립식,1.20%,NaN,NaN,1.20%,NaN,복리,3개월,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,센트럴저축은행,자유적립예금,자유적립식,1.20%,NaN,NaN,1.20%,NaN,복리,3개월이상 ~ 6개월이내,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,아이비케이저축은행,참~똑똑한 IBK저축은행 신용부금,정액적립식,2.00%,NaN,NaN,2.00%,NaN,복리,3개월이상 ~ 6개월이내,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 기간별 실질 이자율 계산하도록

In [34]:
df_short['기간'].unique()

array(['31일', '6개월이내', '100일', '200일', '1개월이상 ~ 6개월이내', '3개월이상 ~ 6개월이내',
       '1개월이상 ~ 3개월이내', '1개월', '3개월', '100일~360일 (일일적금)', '1개월이내', '2개월',
       '5개월이상 ~ 6개월이내', ' 6개월', '1개월이상 ~ 2개월이내'], dtype=object)

In [35]:
df_short['기간'] =  df_short['기간'].apply(lambda x : x.replace('100일~360일 (일일적금)', "1개월이상 ~ 6개월이내") if '100일~360일 (일일적금)'in x else x)
df_short['기간'] =  df_short['기간'].apply(lambda x : x.replace('6개월이내', "1개월이상 ~ 6개월이내") if '6개월이내'== x else x)

In [48]:
# 기간 추출하는 코드
# 만들기
# 3개월 이하 상품 / 6개월 이하 상품
short_term = {
    '31일' : '3개월이하',
    '6개월이내' : '6개월미만',
    '100일' : '3개월이하',
    '200일': '6개월미만',
    '3개월이상 ~ 6개월이내': '6개월미만',
     '1개월이상 ~ 3개월이내' : '3개월이하',
     '1개월' : '3개월이하',
    '3개월' : '3개월이하',
     '100일~360일 (일일적금)' : '6개월미만',
    '1개월이내': '3개월이하',
    '2개월': '3개월이하',
    '1개월이상 ~ 2개월이내' : '3개월이하',
    '3개월이하' : '3개월이하',
    '6개월미만' :'6개월미만'
}

# '1개월이상 ~ 6개월이내' 은 row 두개로 나누기

tmp = df_short[df_short['기간'] == '1개월이상 ~ 6개월이내']
tmp['기간'] = '3개월이하'

tmp2 = df_short[df_short['기간'] == '1개월이상 ~ 6개월이내']
tmp2['기간'] = '6개월미만'

tmp = pd.concat([tmp, tmp2])
tmp

#  '5개월이상 ~ 6개월이내' 삭제
df_short = df_short[df_short['기간'] != "5개월이상 ~ 6개월이내"]

# ' 6개월' 삭제
df_short = df_short[df_short['기간'] != " 6개월"]

#
df_short = df_short[df_short['기간'] != '1개월이상 ~ 6개월이내']

df_short = pd.concat([df_short, tmp])

df_short.reset_index(drop = True, inplace = True)

In [49]:
df_short['기간'] = df_short['기간'].apply(lambda x : short_term[x] )

In [51]:
df_short

,금융회사,상품명,적립방식,세전이자율,세후이자율,세후이자(예시),최고우대금리,실질이자율,이자계산방식,기간,...,만기,변동금리,이벤트,기타,나이,환경,신혼,반려인,직장인,여행
0,주식회사 카카오뱅크,카카오뱅크 한달적금,자유적립식,2.50%,NaN,NaN,8.00%,NaN,복리,3개월이하,...,0.1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,3개월이하,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,3개월이하,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,우리은행,N일 적금,자유적립식,2.00%,NaN,NaN,6.00%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,토스뱅크 주식회사,토스뱅크 자유 적금,자유적립식,1.50%,NaN,NaN,2.50%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,예가람저축은행,자유적립예금,자유적립식,1.00%,NaN,NaN,1.00%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,중소기업은행,정기적금[정액적금],정액적립식,3.20%,NaN,NaN,3.20%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,경남은행,정기적금,정액적립식,2.40%,NaN,NaN,2.40%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,융창저축은행,자유적립예금,자유적립식,1.50%,NaN,NaN,1.50%,NaN,복리,6개월미만,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_short.to_csv("./short_term_range_2.csv", encoding = 'utf-8-sig')

In [50]:
bir = 0.035 # 현재 기준금리 3.5% 반영

# 단리계산
def month_real(interest,month):
    if not np.isnan(interest):
        a =  - sum([ -100 / (1 + bir) ** (i/12.0)   for i in range(0,month)]) #만기시까지 지급한 금액의 현재가치
        b = 100.0 * month + np.sum([ 100 * (interest * 0.01) * i / 12.0   for i in range(1,month + 1)]) #만기후 수령액의 미래가치
        b = b / (1.0125)**(month/12.0) # 만기후 수령액의 현재가치

        return ((b/a)**(12.0/month) - 1) * 100
    else:
        return 0

# 복리계산
def month_complex_real(interest,month):
    if interest != np.nan:
        a =  - sum([ -100 / (1+ bir) ** (i/12.0)   for i in range(0,month)])
        b = np.sum([ 100 *   (1+ ((interest * 0.01) / 12.0)) ** (i)  for i in range(1,month + 1)])
        b = b / (1 + bir)**(month/12.0)
        return ((b/a)**(12.0/month) - 1) * 100
    else:
        return 0